# Gene set enrichment analysis

**Goal:** To detect modest but coordinated changes in prespecified sets of related genes (i.e. those genes in the same pathway or share the same GO term).

1. Ranks all genes based using DE association statistics. In this case we used the p-value scores to rank genes. logFC returned error -- need to look into this.
2. An enrichment score (ES) is defined as the maximum distance from the middle of the ranked list. Thus, the enrichment score indicates whether the genes contained in a gene set are clustered towards the beginning or the end of the ranked list (indicating a correlation with change in expression). 
3. Estimate the statistical significance of the ES by a phenotypic-based permutation test in order to produce a null distribution for the ES( i.e. scores based on permuted phenotype)

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import random
import seaborn as sns
import rpy2.robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

sys.path.append("../")
from functions import utils

from numpy.random import seed
randomState = 123
seed(randomState)

2020-05-27 11:37:49,829 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))



In [2]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(),"../"))

config_file = os.path.abspath(os.path.join(base_dir,
                                           "Rank_pathways",
                                           "init_config.tsv"))
params = utils.read_config(config_file)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
# Load params
local_dir = params["local_dir"]
dataset_name = params['dataset_name']
num_runs = params['num_simulated']
project_id = params['project_id']

rerun_template = True
rerun_simulated = False

### Install R libraries

In [4]:
%%R
# Select 59
# Run one time
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install('clusterProfiler')
#BiocManager::install("biomaRt")
#install.packages("devtools")
#BiocManager::install("GSA")

2020-05-27 11:37:54,740 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:37:54,741 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package(s) 'clusterProfiler'

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:37:55,620 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/clusterProfiler_3.14.3.tar.gz'

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:37:55,663 [26694] WARNING  py.warnings:110: [JupyterRequire] /h

Selection: 59


2020-05-27 11:38:43,101 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.revolutionanalytics.com/src/contrib/devtools_2.3.0.tar.gz'

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:38:43,543 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/octet-stream'
  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:38:43,544 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 375227 bytes (366 KB)

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:38:43,810 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3

--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Ghent) [https]
 8: Brazil (BA) [https]
 9: Brazil (PR) [https]
10: Brazil (RJ) [https]
11: Brazil (SP 1) [https]
12: Brazil (SP 2) [https]
13: Bulgaria [https]
14: China (Hong Kong) [https]
15: China (Guangzhou) [https]
16: China (Lanzhou) [https]
17: China (Shanghai) [https]
18: Colombia (Cali) [https]
19: Costa Rica [https]
20: Denmark [https]
21: East Asia [https]
22: Ecuador (Cuenca) [https]
23: Ecuador (Quito) [https]
24: France (Lyon 1) [https]
25: France (Lyon 2) [https]
26: France (Marseille) [https]
27: Germany (Erlangen) [https]
28: Germany (Münster) [https]
29: Germany (Regensburg) [https]
30: Greece [https]
31: Hungary [https]
32: Iceland [https]
33: Indonesia (Jakarta) [https]
34: Italy (Padua) [https]
35: J

In [5]:
%%R
suppressWarnings(library("clusterProfiler"))
suppressWarnings(library("org.Hs.eg.db"))
suppressWarnings(library("DOSE"))
suppressWarnings(library("biomaRt"))
suppressWarnings(library("fgsea"))
suppressWarnings(library("GSA"))

2020-05-27 11:38:52,468 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Registered S3 method overwritten by 'enrichplot':
  method               from
  fortify.enrichResult DOSE

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:38:52,499 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: clusterProfiler v3.14.3  For help: https://guangchuangyu.github.io/software/clusterProfiler

If you use clusterProfiler in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Yanyan Han, Qing-Yu He. clusterProfiler: an R package for comparing biological themes among gene clusters. OMICS: A Journal of Integrative Biology. 2012, 16(5):284-287.

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:38:52,503 [26694] WARNING  py.warnings:110: [Jupyt

### Get pathway enrichment for template experiment

In [6]:
# Load Hallmark pathways database used by Powers et. al.
# https://github.com/CostelloLab/GSEA-InContext/blob/master/data/gene_sets/hallmarks.gmt
hallmark_DB_file = os.path.join(
    local_dir,
    "hallmark_DB.gmt")

In [10]:
template_DE_stats_file = os.path.join(
    local_dir,
    "DE_stats",
    "DE_stats_template_data_"+project_id+"_real.txt")

In [11]:
%%R -i template_DE_stats_file -o gene_id_mapping

source('../functions/GSEA_analysis.R')

gene_id_mapping <- get_ensembl_symbol_mapping(template_DE_stats_file)

2020-05-27 11:39:50,154 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Cache found

  warnings.warn(x, RRuntimeWarning)

2020-05-27 11:39:50,511 [26694] WARNING  py.warnings:110: [JupyterRequire] /home/alexandra/anaconda3/envs/ranked_pathways/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)



In [12]:
# Set ensembl id as index
gene_id_mapping.set_index("ensembl_gene_id", inplace=True)
print(gene_id_mapping.shape)
gene_id_mapping.head()

(57210, 1)


,hgnc_symbol
ensembl_gene_id,
ENSG00000002330,BAD
ENSG00000003137,CYP26B1
ENSG00000003249,DBNDD1
ENSG00000004799,PDK4
ENSG00000006062,MAP3K14


In [13]:
# Replace ensembl ids with gene symbols
utils.replace_ensembl_ids(template_DE_stats_file,
                          gene_id_mapping)

In [33]:
# Shuffle ranking
# Create 5 shuffled datasets
for r in range(5):
    template_DE_stats = pd.read_csv(
    template_DE_stats_file,
    header=0,
    sep='\t',
    index_col=0)
    
    shuffled_template_DE_stats_file = os.path.join(
    local_dir,
    "DE_stats",
    "DE_stats_template_data_"+project_id+"_shuffled_"+str(r)+".txt")
    
    shuffled_template_DE_stats = template_DE_stats.iloc[np.random.permutation(len(template_DE_stats))]
    shuffled_template_DE_stats.index = template_DE_stats.index
    
    # Save 
    shuffled_template_DE_stats.to_csv(shuffled_template_DE_stats_file, float_format='%.5f', sep='\t')

In [77]:
# Test shuffled files
r = 2
shuffled_template_DE_stats_file = os.path.join(
    local_dir,
    "DE_stats",
    "DE_stats_template_data_"+project_id+"_shuffled_"+str(r)+".txt")

In [101]:
#%%R
#wpgmtfile <- system.file("extdata/wikipathways-20180810-gmt-Homo_sapiens.gmt", package="clusterProfiler")
#wp2gene <- read.gmt(wpgmtfile)
#wpid2gene <- wp2gene %>% dplyr::select(wpid, gene) #TERM2GENE
#wp2gene

                                                                                                                                                             ont
1                                                                                      B Cell Receptor Signaling Pathway%WikiPathways_20180810%WP23%Homo sapiens
2                                                                                      B Cell Receptor Signaling Pathway%WikiPathways_20180810%WP23%Homo sapiens
3                                                                                      B Cell Receptor Signaling Pathway%WikiPathways_20180810%WP23%Homo sapiens
4                                                                                      B Cell Receptor Signaling Pathway%WikiPathways_20180810%WP23%Homo sapiens
5                                                                                      B Cell Receptor Signaling Pathway%WikiPathways_20180810%WP23%Homo sapiens
6                                 

In [103]:
%%R -i template_DE_stats_file -i hallmark_DB_file
# Read in data
DE_stats_data <- read.table(template_DE_stats_file, sep="\t", header=TRUE, row.names=NULL)

# Sort genes by feature 1

# feature 1: numeric vector
# 5: p-values
# 6: adjusted p-values
# 2: logFC
rank_genes <- as.numeric(as.character(DE_stats_data[,4]))

#print(head(rank_genes))

# feature 2: named vector of gene ids
# Remove version from gene id
DE_stats_data[,1] <- gsub("\\..*","", DE_stats_data[,1])

names(rank_genes) <- as.character(DE_stats_data[,1])

print(head(rank_genes))

## feature 3: decreasing order
rank_genes = sort(rank_genes, decreasing = TRUE)

#pathway_DB_data <- read.gmt(hallmark_DB_file)
#pathway_DB_data <- GSA.read.gmt(hallmark_DB_file)
#pathway_parsed <- {}
#for (i in 1:length(pathway_DB_data$genesets)){
#pathway_parsed[pathway_DB_data$geneset.name[i]] <- as.list(pathway_DB_data$genesets[i])
#}
#print(head(pathway_DB_data))
# GSEA is a generic gene set enrichment function
# Different backend methods can be applied depending on the 
# type of annotations
# Here we will use fgsea
#enrich_pathways <- GSEA(geneList=rank_genes, 
#                        TERM2GENE=pathway_DB_data,
#                        nPerm=100000,
#                        by='fgsea',
#                        verbose=T)
#enrich_pathways <- fgsea(pathways=pathway_parsed,
#                         stats=rank_genes,
#                         nperm=100000)
print(enrich_pathways)
#plotEnrichment(pathway_parsed[["HALLMARK_P53_PATHWAY"]], stats=rank_genes, gseaParam = 1, ticksSize = 0.2)
#barplot(sort(rank_genes, decreasing = T))

 SNRK-AS1     ITIH5     HYOU1     GATA6  KIAA0040     PXDC1 
-20.97634 -20.73338  17.87831 -17.37923 -17.08163 -17.04034 
12345678910111213141516171819202122232425262728293031323334353637383940414243444546474849501
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
                                       pathway         pval         padj
 1:                      HALLMARK_ADIPOGENESIS 1.736692e-02 2.226529e-02
 2:               HALLMARK_ALLOGRAFT_REJECTION 8.371178e-05 1.674236e-04
 3:                 HALLMARK_ANDROGEN_RESPONSE 1.718096e-01 1.789683e-01
 4:                      HALLMARK_ANGIOGENESIS 4.056403e-03 5.794862e-03
 5:                   HALLMARK_APICAL_JUNCTION 2.091263e-05 5.090333e-05
 6:                    HALLMARK_APICAL_SURFACE 3.191740e-03 4.999059e-03
 7:                         HALLMARK_APOPTOSIS 2.092444e-05 5.090333e-05
 8:              HALLMARK_BILE_ACID_METABOLISM 3.526819e-03 5.186

*fgsea* 

For each of the input pathways, an ES value is calculated. Next, a number
of random gene sets of the same size are generated, and for each of them
an ES value is calculated. Then a P-value is estimated as the number of
random gene sets with the same or more extreme ES value divided by the
total number of generated gene sets

What is the ES value that is used?
Source code for *fgsea* is available [here](https://rdrr.io/bioc/fgsea/src/R/fgsea.R). This package was based on the method published by [Korotkevich et. al.](https://www.biorxiv.org/content/10.1101/060012v2.full.pdf).
Blog about calulcation is [here](https://www.pathwaycommons.org/guide/primers/data_analysis/gsea/).

In [ ]:
%%R -i template_DE_stats_file -i hallmark_DB_file -o template_enriched_pathways

source('../functions/GSEA_analysis.R')

template_enriched_pathways <- find_enriched_pathways(template_DE_stats_file, hallmark_DB_file)

In [ ]:
print(template_enriched_pathways.shape)
template_enriched_pathways.head()

### Get pathway enrichment for simulated experiments

In [ ]:
# Replace ensembl ids with gene symbols
if rerun_simulated:
    for i in range(num_runs):
        simulated_DE_stats_file = os.path.join(
            local_dir, 
            "DE_stats",
            "DE_stats_simulated_data_"+project_id+"_"+str(i)+".txt")

        utils.replace_ensembl_ids(simulated_DE_stats_file,
                                  gene_id_mapping)

In [ ]:
%%R -i project_id -i local_dir -i hallmark_DB_file -i num_runs -i rerun_simulated

source('../functions/GSEA_analysis.R')

for (i in 0:(num_runs-1)){
    simulated_DE_stats_file <- paste(local_dir, 
                                 "DE_stats/DE_stats_simulated_data_", 
                                 project_id,
                                 "_", 
                                 i,
                                 ".txt",
                                 sep="")
    
    out_file = paste(local_dir, 
                     "GSEA_stats/GSEA_simulated_data_",
                     project_id,
                     "_",
                     i,
                     ".txt", 
                     sep="")
    
    if (rerun_simulated){
        enriched_pathways <- find_enriched_pathways(simulated_DE_stats_file, hallmark_DB_file) 
        #print(head(enriched_pathways))
    
        write.table(enriched_pathways, file = out_file, row.names = T, sep = "\t")
        }
    }

**Check**

Again, we want to compare our ranked pathways found against what was reported in the original publication.

## Get statistics for enriched pathways
Examine the enriched pathways identified from template experiment -- How are these enriched pathways ranked in the simulated experiments?

### Template experiment

In [ ]:
col_to_rank = 'enrichmentScore'

In [ ]:
# Get ranks of template experiment
# Rank pathways by highest enrichment score
template_enriched_pathways['ranking'] = template_enriched_pathways[col_to_rank].rank(ascending = 0) 
template_enriched_pathways = template_enriched_pathways.sort_values(by=col_to_rank, ascending=False)

# Set index to GO ID
template_enriched_pathways.set_index("ID", inplace=True)
print(template_enriched_pathways.shape)
template_enriched_pathways.head()

In [ ]:
# Check that GO IDs are unique
template_enriched_pathways.index.nunique() == len(template_enriched_pathways)

### Simulated experiments

In [ ]:
# Concatenate simulated experiments
simulated_enriched_pathways_all = pd.DataFrame()
for i in range(num_runs):
    simulated_GSEA_file = os.path.join(
        local_dir, 
        "GSEA_stats",
        "GSEA_simulated_data_"+project_id+"_"+str(i)+".txt")
    
    #Read results
    simulated_enriched_pathways = pd.read_csv(
        simulated_GSEA_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # Add ranks of simulated experiment
    simulated_enriched_pathways['ranking'] = simulated_enriched_pathways[col_to_rank].rank(ascending = 0) 
    simulated_enriched_pathways = simulated_enriched_pathways.sort_values(by=col_to_rank, ascending=False)
    
    # Concatenate df
    simulated_enriched_pathways_all = pd.concat([simulated_enriched_pathways_all,
                                       simulated_enriched_pathways])
    
print(simulated_enriched_pathways_all.shape)
simulated_enriched_pathways_all.head()

In [ ]:
simulated_enriched_pathways_stats = simulated_enriched_pathways_all.groupby(['ID'])[['enrichmentScore', 'pvalue', 'ranking']].agg({
    col_to_rank:['mean', 'std','count'],
    'pvalue':['median'],
    'ranking':['median']
})

simulated_enriched_pathways_stats.head()

In [ ]:
# Merge template statistics with simulated statistics
template_simulated_enriched_pathways_stats = template_enriched_pathways.merge(simulated_enriched_pathways_stats, 
                                                                              how='outer',
                                                                              left_index=True,
                                                                              right_index=True)
template_simulated_enriched_pathways_stats.head()

In [ ]:
# Parse columns
median_pval_simulated = template_simulated_enriched_pathways_stats[('pvalue','median')]
median_rank_simulated = template_simulated_enriched_pathways_stats[('ranking','median')]
mean_test_simulated = template_simulated_enriched_pathways_stats[(col_to_rank,'mean')]
std_test_simulated = template_simulated_enriched_pathways_stats[(col_to_rank,'std')]
count_simulated = template_simulated_enriched_pathways_stats[(col_to_rank,'count')]

### Calculations for summary table

In [ ]:
summary = pd.DataFrame(data={'Pathway': template_simulated_enriched_pathways_stats.index,
                             'P-value (Real)': template_simulated_enriched_pathways_stats['pvalue'],
                             'Rank (Real)': template_simulated_enriched_pathways_stats['ranking'],
                             'Test statistic (Real)': template_enriched_pathways[col_to_rank],
                             'Median p-value (simulated)': median_pval_simulated ,
                             'Median rank (simulated)': median_rank_simulated ,
                             'Mean test statistic (simulated)': mean_test_simulated ,
                             'Std deviation (simulated)': std_test_simulated,
                             'Number of experiments (simulated)': count_simulated
                            }
                      )
summary['Z score'] = (summary['Test statistic (Real)'] - summary['Mean test statistic (simulated)'])/summary['Std deviation (simulated)']
summary

In [ ]:
# Save file
summary_file = os.path.join(
        local_dir, 
        "pathway_summary_table.tsv")

summary.to_csv(summary_file, float_format='%.5f', sep='\t')